In [1]:
import cv2
import pickle
import numpy as np
from tensorflow.keras.models import load_model
import time
import tensorflow as tf

# === GPU Setup ===
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU available, running on CPU")

# === Load Model ===
model = load_model("model_final.h5")

# === Class Labels ===
class_dictionary = {0: 'no_car', 1: 'car', 2: 'wrong_parking'}

# === Load Video ===
video = cv2.VideoCapture("car_test_-car2.mp4")

# test function
def alert_wrong_parking(spot_number):
    print(spot_number)





# === Load and Fix carposition.pkl ===
with open('carposition.pkl', 'rb') as f:
    positionList = pickle.load(f)

# Fix and enforce correct format
fixed_list = []
for i, item in enumerate(positionList):
    if isinstance(item, tuple):
        if len(item) == 2:
            if isinstance(item[0], tuple) and isinstance(item[1], int):
                # Already in ((x, y), spot_number) format
                fixed_list.append(item)
            elif isinstance(item[0], int) and isinstance(item[1], int):
                # Old format: (x, y)
                fixed_list.append(((item[0], item[1]), i + 1))
            else:
                print(f"❌ Unrecognized format at index {i}: {item}")
        else:
            print(f"❌ Tuple of unexpected length at index {i}: {item}")
    else:
        print(f"❌ Not a tuple at index {i}: {item}")

positionList = fixed_list
with open('carposition.pkl', 'wb') as f:
    pickle.dump(positionList, f)
print("✅ Cleaned and saved corrected positionList.")

# === Constants ===
width, height = 130, 65
confirmation_duration = 5  # seconds

# === Trackers ===
state_start_time = {}
current_state = {}


alerted_once = []

# === Parking Check Function ===
def checkingCarParking(img):
    imgCrops = []
    spot_info = []
    spaceCounter = 0
    wrongParkingCounter = 0

    # Crop regions from image
    for item in positionList:
        pos, spot_number = item
        x, y = pos
        cropped_img = img[y:y+height, x:x+width]
        imgResized = cv2.resize(cropped_img, (48, 48))
        imgNormalized = imgResized / 255.0
        imgCrops.append(imgNormalized)
        spot_info.append({'position': pos, 'number': spot_number})

    imgCrops = np.array(imgCrops)

    # Predict
    with tf.device('/GPU:0' if gpus else '/CPU:0'):
        predictions = model.predict(imgCrops, verbose=0)

    # Draw results
    for i, spot in enumerate(spot_info):
        pos = spot['position']
        spot_number = spot['number']
        x, y = pos
        intId = np.argmax(predictions[i])
        label = class_dictionary[intId]

        # Update state and confirmation
        if pos not in current_state:
            current_state[pos] = label
            state_start_time[pos] = time.time()

        if label == current_state[pos]:
            elapsed = time.time() - state_start_time[pos]
            confirmed = elapsed >= confirmation_duration
        else:
            current_state[pos] = label
            state_start_time[pos] = time.time()
            confirmed = False

        # Visual style
        if label == 'no_car':
            color = (0, 255, 0) if confirmed else (150, 255, 150)
            thickness = 5
            textColor = (0, 0, 0)
            if confirmed:
                spaceCounter += 1
                if spot_number in alerted_once:
                    alerted_once.remove(spot_number)
        elif label == 'wrong_parking':
            color = (255, 255, 0) if confirmed else (255, 255, 150)
            thickness = 2
            textColor = (255, 255, 255)
            if confirmed:
                wrongParkingCounter += 1
                if spot_number not in alerted_once:
                    alert_wrong_parking(spot_number)
                    alerted_once.append(spot_number)
        else:
            color = (0, 0, 255) if confirmed else (150, 150, 255)
            thickness = 2
            textColor = (255, 255, 255)
            if spot_number in alerted_once:
                alerted_once.remove(spot_number)

        # Draw box
        cv2.rectangle(img, pos, (pos[0]+width, pos[1]+height), color, thickness)

        # Draw label box and text
        font_scale = 0.5
        text_thickness = 1
        textSize = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_thickness)[0]
        textX = x
        textY = y + height - 5
        cv2.rectangle(img, (textX, textY - textSize[1] - 5), (textX + textSize[0] + 6, textY + 2), color, -1)
        cv2.putText(img, label, (textX + 3, textY - 3), cv2.FONT_HERSHEY_SIMPLEX, font_scale, textColor, text_thickness)

        # Draw spot number
        cv2.putText(img, str(spot_number), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Draw summary
    cv2.putText(img, f'Space Count: {spaceCounter}', (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(img, f'Wrong Parking Count: {wrongParkingCounter}', (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# === Main Loop ===
while True:
    if video.get(cv2.CAP_PROP_POS_FRAMES) == video.get(cv2.CAP_PROP_FRAME_COUNT):
        video.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    ret, image = video.read()
    if not ret:
        break

    image = cv2.resize(image, (1280, 720))
    checkingCarParking(image)
    cv2.imshow("Image", image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()


1 Physical GPUs, 1 Logical GPUs
✅ Cleaned and saved corrected positionList.


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
from ultralytics import YOLO

yolo_model = YOLO('parking_spot_detection/yolov8_parking_model/weights/best.pt')


In [ ]:
from flask import Flask, render_template, Response, jsonify
import cv2
import numpy as np
from keras.models import load_model
import time
import tensorflow as tf
import torch
import torch.nn.modules.container as container_module  # Already added before
from send_alert import alert_wrong_parking
from ultralytics import YOLO
from ultralytics.nn.tasks import DetectionModel
from ultralytics.nn.modules.conv import Conv  # Import Conv

# === Allow YOLO Model Classes for Safe Loading (PyTorch >= 2.6) ===
torch.serialization.add_safe_globals([DetectionModel])
torch.serialization.add_safe_globals([container_module.Sequential])
torch.serialization.add_safe_globals([Conv])  # Allow Conv class

# === Initialize Flask App ===
app = Flask(__name__)

# === GPU Configuration for TensorFlow ===
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU available, running on CPU")

# === Load Models ===
classifier_model = load_model('model_final.h5')
yolo_model = YOLO('parking_spot_detection/yolov8_parking_model/weights/best.pt')




# === Class Mapping ===
class_dictionary = {0: 'no_car', 1: 'car', 2: 'wrong_parking'}

# === Parameters and State Tracking ===
confirmation_duration = 5
state_start_time = {}
current_state = {}
alerted_once = []

# === Load Video ===
cap = cv2.VideoCapture('car_test_-car2.mp4')

# === Parking Spot Detection and Classification ===
def checkParkingSpace(img):
    imgCrops = []
    spot_info = []
    spaceCounter = 0
    wrongParkingCounter = 0

    results = yolo_model.predict(img, conf=0.4, show_labels=False, show_conf=False, verbose=False)
    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box
        cropped_img = img[y1:y2, x1:x2]
        if cropped_img.shape[0] == 0 or cropped_img.shape[1] == 0:
            continue
        imgResized = cv2.resize(cropped_img, (48, 48))
        imgNormalized = imgResized / 255.0
        imgCrops.append(imgNormalized)
        spot_info.append({'bbox': box, 'number': i + 1})

    if not imgCrops:
        return img, 0, 0, 0

    imgCrops = np.array(imgCrops)

    with tf.device('/GPU:0' if gpus else '/CPU:0'):
        predictions = classifier_model.predict(imgCrops, verbose=0)

    for i, spot in enumerate(spot_info):
        x1, y1, x2, y2 = spot['bbox']
        spot_number = spot['number']
        intId = np.argmax(predictions[i])
        label = class_dictionary[intId]

        pos_key = (x1, y1, x2, y2)
        if pos_key not in current_state:
            current_state[pos_key] = label
            state_start_time[pos_key] = time.time()

        if label == current_state[pos_key]:
            elapsed = time.time() - state_start_time[pos_key]
            confirmed = elapsed >= confirmation_duration
        else:
            current_state[pos_key] = label
            state_start_time[pos_key] = time.time()
            confirmed = False

        if label == 'no_car':
            color = (0, 255, 0) if confirmed else (150, 255, 150)
            thickness = 5
            textColor = (0, 0, 0)
            if confirmed:
                spaceCounter += 1
                if spot_number in alerted_once:
                    alerted_once.remove(spot_number)
        elif label == 'wrong_parking':
            color = (255, 255, 0) if confirmed else (255, 255, 150)
            thickness = 2
            textColor = (255, 255, 255)
            if confirmed:
                wrongParkingCounter += 1
                if spot_number not in alerted_once:
                    alert_wrong_parking(spot_number)
                    alerted_once.append(spot_number)
        else:
            color = (0, 0, 255) if confirmed else (150, 150, 255)
            thickness = 2
            textColor = (255, 255, 255)
            if spot_number in alerted_once:
                alerted_once.remove(spot_number)

        cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
        cv2.putText(img, label, (x1 + 3, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, textColor, 1)
        cv2.putText(img, str(spot_number), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    totalSpaces = len(spot_info)
    occupied_spaces = totalSpaces - spaceCounter - wrongParkingCounter

    cv2.putText(img, f'Space Count: {spaceCounter}', (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(img, f'Wrong Parking Count: {wrongParkingCounter}', (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    return img, spaceCounter, wrongParkingCounter, occupied_spaces

# === Frame Generator for MJPEG Streaming ===
def generate_frames():
    while True:
        if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

        success, img = cap.read()
        if not success:
            break

        img = cv2.resize(img, (1280, 720))
        img, free_spaces, wrong_parking_spaces, occupied_spaces = checkParkingSpace(img)

        ret, buffer = cv2.imencode('.jpg', img)
        img = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + img + b'\r\n')

# === Flask Routes ===
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/space_count')
def space_count():
    success, img = cap.read()
    if success:
        img = cv2.resize(img, (1280, 720))
        _, free_spaces, wrong_parking_spaces, occupied_spaces = checkParkingSpace(img)
        return jsonify(free=free_spaces, wrong=wrong_parking_spaces, occupied=occupied_spaces)
    return jsonify(free=0, wrong=0, occupied=0)

# === Run Flask App ===
if __name__ == "__main__":
    app.run(debug=True)


ModuleNotFoundError: No module named 'ultralytics.yolo.models'